In [3]:
import os
import sys
import urllib.request
import json
import pickle
import pandas as pd
import numpy as np
import pandas as pd
import time
import pickle
import gc
import os
import itertools
from datetime import datetime , date
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [4]:
os.chdir("제6회 L.POINT Big Data Competition/")

online_df = pd.read_csv("clear_merged_online_df.csv")
audience = pd.read_csv("audience_with_group.csv")
base = pd.read_csv("normal_with_group.csv")

In [5]:
dataset_purchase = pd.read_csv("buying_list_clac3.csv")
naver_cat_matching = pd.read_csv("product_table.csv", encoding="euc-kr")
asso_df = pd.read_csv("clac_nm3_rules_list.csv")
asso_df.drop(asso_df.columns[1], axis=1, inplace = True)

In [18]:
"""mappers = {
    "cat3_to_cat1" : online_df[["clac_nm1","clac_nm3"]].dropna().drop_duplicates(["clac_nm1","clac_nm3"], keep ="first").set_index("clac_nm3").to_dict().get("clac_nm1"),
    "cat1_to_naver" : naver_cat_matching.set_index("clac_nm1").to_dict().get("naver_cat"),
    "clac_nm3" : sorted(online_df.clac_nm3.dropna().unique().tolist())
}

with open("mappers.pickle", "wb") as f:
    pickle.dump(mappers, f, protocol=pickle.HIGHEST_PROTOCOL)"""

In [38]:
# 기존고객들 정보를 clnt_id별로 나누어 array형태 pickle파일로 저장
# 이후에 정보를 불러올때 전체파일을 불러오면 시간상 손해가 많아 각각 나눠놓음
"""
for i in audience.clnt_id:
    if not os.path.exists("./grouped_info_by_clnt_id/"):
        os.mkdir("./grouped_info_by_clnt_id/")
    tmp = audience.loc[audience.clnt_id == i].values
    group = tmp[0][-1].astype(int)
    path = "./grouped_info_by_clnt_id/"
    file = "grouped_clnt_info_{}.pickle".format(i)
    with open(os.path.join(path,file), "wb") as f:
        pickle.dump(tmp, f, protocol=pickle.HIGHEST_PROTOCOL)

for i in base.clnt_id:
    if not os.path.exists("./grouped_info_by_clnt_id/"):
        os.mkdir("./grouped_info_by_clnt_id/")
    tmp = base.loc[base.clnt_id == i].values
    group = tmp[0][-1].astype(int)
    path = "./grouped_info_by_clnt_id/"
    file = "grouped_clnt_info_{}.pickle".format(i)
    with open(os.path.join(path,file), "wb") as f:
        pickle.dump(tmp, f, protocol=pickle.HIGHEST_PROTOCOL)
"""

'\nfor i in audience.clnt_id:\n    if not os.path.exists("./grouped_info_by_clnt_id/"):\n        os.mkdir("./grouped_info_by_clnt_id/")\n    tmp = audience.loc[audience.clnt_id == i].values\n    group = tmp[0][-1].astype(int)\n    path = "./grouped_info_by_clnt_id/"\n    file = "grouped_clnt_info_{}.pickle".format(i)\n    with open(os.path.join(path,file), "wb") as f:\n        pickle.dump(tmp, f, protocol=pickle.HIGHEST_PROTOCOL)\n\nfor i in base.clnt_id:\n    if not os.path.exists("./grouped_info_by_clnt_id/"):\n        os.mkdir("./grouped_info_by_clnt_id/")\n    tmp = base.loc[base.clnt_id == i].values\n    group = tmp[0][-1].astype(int)\n    path = "./grouped_info_by_clnt_id/"\n    file = "grouped_clnt_info_{}.pickle".format(i)\n    with open(os.path.join(path,file), "wb") as f:\n        pickle.dump(tmp, f, protocol=pickle.HIGHEST_PROTOCOL)\n'

In [40]:
with open("holiday_list.pickle", "rb") as f:
    holiday_list = pickle.load(f)
    
with open("naver_shopping_insight.pickle", "rb") as f:
    naver_shopping_insight = pickle.load(f)

with open("lstm_trend_dict.pickle", "rb") as f:
    trend_dict = pickle.load(f)
    
with open("clac_nm_list.pickle", 'rb') as f:
    clac_nm_list = pickle.load(f)
    
dnn = tf.keras.models.load_model("dnn_model.h5")

#sorted_cat = sorted(naver_shopping_insight.items(), key=(lambda x : x[1]), reverse=True)

cat3_to_cat1 = online_df[["clac_nm1","clac_nm3"]].dropna().drop_duplicates(["clac_nm1","clac_nm3"], keep ="first").set_index("clac_nm3").to_dict().get("clac_nm1")
cat1_to_naver = naver_cat_matching.set_index("clac_nm1").to_dict().get("naver_cat")
clac_nm3 = sorted(online_df.clac_nm3.dropna().unique().tolist())

In [41]:
#buying_list = buying_list.iloc[~buying_list.apply(lambda x : x[1:].sum() == x.iloc[-1], axis=1).values]
#buying_list = buying_list.iloc[:,0:-1]
#buying_list.to_csv("buying_list_clac3.csv", index=False)

In [42]:
# Using Naver Shopping Insight

client_id = "aDa9tsBBqchLu4JZ6awC"
client_secret = "XP6uLmitVE"

def getTrend(date="2019-10-01"):
    body1 = '{"startDate":"'+str(date)+'","endDate":"'+str(date)+'","timeUnit":"date","category":[{"name":"패션의류","param":["50000000"]},{"name":"패션잡화","param":["50000001"]},{"name":"화장품/미용","param":["50000002"]}]}'
    body2 = '{"startDate":"'+str(date)+'","endDate":"'+str(date)+'","timeUnit":"date","category":[{"name":"디지털/가전","param":["50000003"]},{"name":"가구/인테리어","param":["50000004"]},{"name":"출산/육아","param":["50000005"]}]}'
    body3 = '{"startDate":"'+str(date)+'","endDate":"'+str(date)+'","timeUnit":"date","category":[{"name":"식품","param":["50000006"]},{"name":"스포츠/레저","param":["50000007"]},{"name":"생활/건강","param":["50000008"]}]}'
    body4 = '{"startDate":"'+str(date)+'","endDate":"'+str(date)+'","timeUnit":"date","category":[{"name":"패션의류","param":["50000000"]},{"name":"디지털/가전","param":["50000003"]},{"name":"생활/건강","param":["50000008"]}]}'
    
    url = "https://openapi.naver.com/v1/datalab/shopping/categories"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    request.add_header("Content-Type","application/json")
    
    response1 = urllib.request.urlopen(request, data=body1.encode("utf-8"))
    response2 = urllib.request.urlopen(request, data=body2.encode("utf-8"))
    response3 = urllib.request.urlopen(request, data=body3.encode("utf-8"))
    response4 = urllib.request.urlopen(request, data=body4.encode("utf-8"))
    response_list = [response1, response2, response3]
    
    # 각 body 별 비율 계산
    body_rate = []
    rescode = response4.getcode()
    response_body = json.loads(response4.read().decode('utf-8'))
    for i in range(0,3):
        body_rate.append(response_body["results"][i]["data"][0]["ratio"])
    body_rate = [i/max(body_rate) for i in body_rate]
    
    search_rate_dict = {}
    num=0
    for res in response_list:
        rescode = res.getcode()
        if (rescode==200):
            response_body = res.read().decode('utf-8')
            response_body = json.loads(response_body)
            rate = body_rate[num]
            for i in range(0,3):
                search_rate = response_body["results"][i]["data"][0]["ratio"]*rate
                search_rate_dict[response_body["results"][i]["title"]] = search_rate
        else:
            print("Error Code:" + rescode)
        num+=1
    search_rate_dict["기타"]=0
    
    #print(response_result)
    return search_rate_dict

search_rate_dict = getTrend()

with open("naver_shopping_insight.pickle", "wb") as f:
    pickle.dump(search_rate_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
    
naver_shopping_insight = getTrend()

In [ ]:
for i in range(1,7):
    globals()["g{}_weight".format(i)] = pd.DataFrame({"col" : col, "weight":1})

In [14]:
g1_weight = g2_weight = g3_weight = g4_weight = g5_weight = g6_weight = pd.DataFrame({"col" : col, "weight":1})

In [12]:
del[g1_weight,g2_weight,g3_weight,g4_weight,g5_weight,g6_weight ]

In [5]:
col = list(audience.columns[1:-1])

In [43]:
# Using KNN
"""group1: time_to_last, last_access
group2: return_rate, src_PUSH
group3: action_0, action_2, return_rate
group4: tot_sess_cnt, pag_view_mean, src_WEBSITE
group5: tot_sess_cnt, return_rate, action_0
group6: action_2, tot_sess_cnt"""

# 각 그룹별 weight 수정 예정
# 각 normal 그룹에서 구매율 class label로 해서 중요변수 도출?
col = list(audience.columns[1:-1])
g1_weight = pd.DataFrame({"col" : col, "weight":1})
g2_weight = pd.DataFrame({"col" : col, "weight":1})
g3_weight = pd.DataFrame({"col" : col, "weight":1})
g4_weight = pd.DataFrame({"col" : col, "weight":1})
g5_weight = pd.DataFrame({"col" : col, "weight":1})
g6_weight = pd.DataFrame({"col" : col, "weight":1})
g1_weight.iloc[[col.index("time_to_last"), col.index("last_access")], 1] = 0.8
g2_weight.iloc[[col.index("return_rate"), col.index("src_PUSH")], 1] = 0.8
g3_weight.iloc[[col.index("action_0"), col.index("action_2"), col.index("return_rate")], 1] = 0.8
g4_weight.iloc[[col.index("tot_sess_cnt"), col.index("pag_view_mean"), col.index("src_WEBSITE")], 1] = 0.8
g5_weight.iloc[[col.index("tot_sess_cnt"), col.index("return_rate"), col.index("action_0")], 1] = 0.8
g6_weight.iloc[[col.index("action_2"), col.index("tot_sess_cnt")], 1] = 0.8

weight_list_by_group = {1: list(g1_weight.weight),
                       2: list(g2_weight.weight),
                       3: list(g3_weight.weight),
                       4: list(g4_weight.weight),
                       5: list(g5_weight.weight),
                       6: list(g6_weight.weight),}

def similar_user(user_info, user_group, n_neighbor):
    # user_info, user_group => 함수내에서 자동으로 input
    
    weight_list = weight_list_by_group.get(user_group)
    calc_dist = base.iloc[:,1:-1].apply(lambda x :np.linalg.norm(np.multiply(weight_list,(x -  user_info))), axis=1).reset_index(name = "dist")
    top_30_similarity = calc_dist.sort_values(by = "dist").iloc[:n_neighbor,0]
    return base.iloc[top_30_similarity,0].tolist()

In [44]:
base = base[base["clnt_id"].isin(list(dataset_purchase.clnt_id))].reset_index(drop=True)

In [45]:
def similar_user(user_info, user_group, n_neighbor):
    # user_info, user_group => 함수내에서 자동으로 input
    
    weight_list = weight_list_by_group.get(user_group)
    calc_dist = base.iloc[:,1:-1].apply(lambda x :np.linalg.norm(np.multiply(weight_list,(x -  user_info))), axis=1).reset_index(name = "dist")
    top_30_clnt = calc_dist.sort_values(by = "dist").iloc[:n_neighbor,0]
    top_30_similarity = calc_dist.sort_values(by = "dist").iloc[:n_neighbor,1]
    top_30 = pd.DataFrame({"clnt_id": base.iloc[top_30_clnt,0], "dist": top_30_similarity}).reset_index(drop=True)
    return top_30

def multiplier(weight, array):
    return weight*array

def similar_user_weight(top_30):
    # dataset_purchase에 붙여서 가중치 계산
    dataset_purchase_top = dataset_purchase[dataset_purchase["clnt_id"].isin(list(top_30.clnt_id))].reset_index(drop=True)
    top_30["weight"] = 1 / (top_30.dist+(max(list(top_30.dist))-min(list(top_30.dist)))*2)
    top_30 = top_30.drop("dist", axis=1)
    top_30_df = pd.merge(top_30, dataset_purchase_top, on="clnt_id", how="left")
    top_30_df = top_30_df.apply(lambda x : multiplier(x['weight'], x[2:]),axis=1)
    weight_knn = top_30_df.sum(axis=0)
    return weight_knn

In [132]:
# Using LSTM 

def get_recent_sales():
    with open("sales_by_day.pickle", "rb") as f:
        sales = pickle.load(f)
    recent_sales = pd.DataFrame(sales).iloc[:,-7:]
    return recent_sales

def get_recent_time_info(start_time = '20190924', now_time = '20191001'):
    time_info = pd.DataFrame({"time" : pd.date_range(start=start_time, end=now_time)})

    time_info["is_holiday"] = np.where(time_info.isin(pd.to_datetime(holiday_list)), 1, 0)
    time_info["is_weekend"] = np.where(time_info.time.dt.weekday.isin(range(0,5)), 0, 1)

    time_info["holiday_tomorrow"] = time_info.is_holiday.shift(-1, fill_value=0)
    time_info["weekend_tomorrow"] = time_info.is_weekend.shift(-1, fill_value=0)

    time_info.drop(["is_holiday", "is_weekend"], axis=1, inplace=True)
    
    return time_info

def predict_trend_by_lstm():
    import gc
    with open("clac_nm_list.pickle", 'rb') as f:
        clac_nm_list = pickle.load(f)
        
    recent_sales = get_recent_sales()
    time_info = get_recent_time_info()
    yesterday_sales = get_recent_sales().iloc[:,-1].values
    expected_sales_list = []
    
    for i in range(recent_sales.shape[0]):
        # print("Expecting Today's {} Sales ".format(clac_nm_list[i]))
        sales = pd.concat([recent_sales.iloc[i:i+1].T.reset_index(drop = True), time_info.iloc[:-1,1:]], axis=1).values.reshape(1,7,3)
        modelname = os.curdir + "/models/lstm_model_{}.h5".format(clac_nm_list[i].replace("/","_"))
        model = tf.keras.models.load_model(modelname)
        expected_sales = np.round(model.predict(sales)[0][0])
        expected_sales_list.append(expected_sales)
        
    sales_diff = yesterday_sales - np.array(expected_sales_list)
    
    del [recent_sales, time_info]
    gc.collect()
    
    return sales_diff, expected_sales_list

sales_diff, expected_sales_list = predict_trend_by_lstm()
trend_dict = pd.DataFrame({"clac_nm1":clac_nm_list, "sales":np.where(sales_diff > 0, 1.1, np.where(sales_diff==0, 1, 0.9))}).set_index("clac_nm1").to_dict().get("sales")

with open("lstm_trend_dict.pickle", "wb") as f:
    pickle.dump(trend_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'clac_nm_list' is not defined

In [14]:
# Association Rule Discovery
def split_item(item_in_brace):
    return item_in_brace.strip("{}").split(",")

asso_df['lhs'] = asso_df.lhs.apply(lambda x : split_item(x))
asso_df['rhs'] = asso_df.rhs.apply(lambda x : split_item(x))
asso_df['lhs_length'] = asso_df.lhs.apply(lambda x : len(x))

asso_df = asso_df.sort_values(by = ["lhs_length","confidence"], ascending = [1,0]).reset_index(drop = True)

In [16]:
asso_df.to_csv("association_rule_df.csv", index = False)

In [182]:
# 연관된 추천상품 찾기
def find_asso(recommend):
    if (type(recommend)!=list):
        recommend = recommend.to_list()
    asso_list = []
    asso_conf = []
    for i in range(len(recommend)):
        for j in range(len(asso_df[asso_df.lhs_length==1])):
            if (recommend[i] == asso_df.lhs[j][0]):
                asso_list.append(asso_df.rhs[j][0])
                asso_conf.append(asso_df.confidence[j])
    recommend_permu = list(map(','.join, itertools.permutations(recommend, 2)))
    for i in range(len(recommend_permu)):
        for j in range(len(asso_df[asso_df.lhs_length>1])):
            if (recommend_permu[i].split(',')[0] in asso_df.lhs[j] and recommend_permu[i].split(',')[1] in asso_df.lhs[j]):
                asso_list.append(asso_df.rhs[j][0])
                asso_conf.append(asso_df.confidence[j])
    asso_result = pd.DataFrame({"product": asso_list, "confidence": asso_conf}).sort_values(by="confidence", ascending=False)
    asso_result = asso_result.drop_duplicates(['product'], keep='first').reset_index(drop=True)
    return asso_result

In [196]:
# online_df[online_df.action_type == 6].clac_nm1.map(cat1_to_naver).value_counts()

#naver_shopping_insight

def naver_scaler(x):
    return 1/200*x + 1/2

def get_user_info(clnt_id):
    try:
        path = "./grouped_info_by_clnt_id/"
        file = "grouped_clnt_info_{}.pickle".format(clnt_id)
        with open(os.path.join(path,file), "rb") as f:
            user_log_summary = pickle.load(f)
        
        return user_log_summary
    except FileNotFoundError:
        return 0

def recommender(clnt_id, n_neighbor = 30):
    user_log_summary = get_user_info(clnt_id)
    weight = pd.Series([0]*len(clac_nm3))
    min_max_scaler = MinMaxScaler()

    #-------------------- 고객정보가 필요하지 않은 부분 ---------------
    #1. 네이버 API를 통해 쇼핑인사이트의 각 카테고리별 상대적 검색비중을 확인
    #naver_search_cat = [x[0] for x in sorted(naver_shopping_insight.items(), key=(lambda x : x[1]), reverse=True)]
    #naver_search_val = [x[1] for x in sorted(naver_shopping_insight.items(), key=(lambda x : x[1]), reverse=True)]
    print("1) Naver Trend", end="")
    weight_naver = pd.Series(clac_nm3).map(cat3_to_cat1).map(cat1_to_naver).map(naver_shopping_insight)
    weight_naver_scaling = naver_scaler(weight_naver)
    print("\t...Success!")
    
    #2. 대분류 lstm
    print("2) LSTM", end="")
    weight_lstm = pd.Series(clac_nm3).map(cat3_to_cat1).map(trend_dict)
    print("\t...Success!")
    
    #-------------------- 고객정보 필요한 부분 ------------------------
    #3. knn - 비슷한 사람 - 소분류 가중치
    print("3) KNN", end="")
    user_info = user_log_summary[0][1:-1]
    user_group = user_log_summary[0][-1].astype(int)
    weight_knn = similar_user_weight(similar_user(user_info, user_group, n_neighbor))
    weight_knn_scaling = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(weight_knn)))[0]
    print("\t...Success!")
        
    #4. dnn - 행동패턴을 통해 구매할 소분류 예측
    print("4) DNN", end="")
    weight_dnn = pd.Series(dnn.predict(np.expand_dims(list(user_info), axis=0))[0])
    weight_dnn_scaling = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(weight_dnn)))[0]
    print("\t...Success!")
    
    # 최종 가중치 계산 및 추천상품 선정
    final_weight = weight_lstm*(weight_naver_scaling + weight_knn_scaling + weight_dnn_scaling)
    final_result_df = pd.DataFrame({"product": clac_nm3, "weight": final_weight}).sort_values(by="weight", ascending=False)
    final_result = final_result_df.iloc[0:9,0].reset_index(drop=True)
    
    # 5. Association Rule Discovery - 추천된 상품과 함께사면 좋을 음식 추천
    # 음식 아닌 상품 + 음식 + 함께사면 좋을 음식
    #not_food = final_result[final_result.map(cat3_to_cat1).map(cat1_to_naver)!="식품"]
    asso_list = find_asso(final_result)["product"]
    for prod in asso_list:
        if (prod not in list(final_result)):
            final_result[9] = prod
            break
    if (len(final_result)==9):
        final_result[9] = final_result_df.iloc[10,0]
    return final_result


In [197]:
recommender(1)

1) Naver Trend	...Success!
2) LSTM	...Success!
3) KNN	...Success!
4) DNN	...Success!


0                                  Chicken Eggs
1                                        Ramens
2    Other General Stationary / Office Supplies
3                                General Snacks
4                          Women's Trench Coats
5                                     Necklaces
6                                    Fresh Milk
7                               Boy's Sleepwear
8                  Women's Golf T-shirts / Tops
9                                   Corn Snacks
Name: product, dtype: object